In [5]:
#--importing the required python packages
import json as js
import pandas as pd
import psycopg2 as pg
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#--setting up json data file location
file_loc = "C:\\Data\\data.json"

#--reading data from json data file and inserting it into the pandas dataframe
jdf = pd.read_json(file_loc)

#--checking total incoming row count in the pandas dataframe
total_incoming_row_count = len(jdf)

#--printing total row count in the dataframe
print(" ")
print("Total incoming row count : ",total_incoming_row_count)

#--declaring blank list
data_ins_set = []

#--declaring variable with sql statement to clean-up the matching_data table 
del_sql_stm = "TRUNCATE TABLE public.Stage_matching_data RESTART IDENTITY CASCADE;"

#--declaring variable with sql statement to insert data into matching_data table
ins_sql_stm = "INSERT INTO public.Stage_matching_data(display_name,age,job_title ,height_in_cm,city_name,city_lat,city_lon ,main_photo ,\
pokemon_catch_rate,cats_owned,likes_cats,religion) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"

#--declaring variable with sql statement to check the data count from matching_data table
cnt_sql_stm = "SELECT 1 FROM Stage_matching_data;"

#--opening json data file for data extraction
with open(file_loc) as data_file:
    
    #--loading data into data variable for processing
    data = js.load(data_file)
    
    #--loop to go through full data set
    for v in data.values():
        
        #--loop to go through each data row
        for i in range(0, total_incoming_row_count):
            
            #--to check the key display_name
            if "display_name" in v[i]:
                display_name = v[i]["display_name"]
            else:
                display_name = None
            
            #--to check the key age
            if  "age" in v[i]:
                age = v[i]["age"]
            else:
                age = None
            
            #---to check the key job_title
            if "job_title" in v[i]:
                job_title = v[i]["job_title"]
            else:
                job_title = None
            
            #--to check the key height_in_cm
            if "height_in_cm" in v[i]:
                height_in_cm = v[i]["height_in_cm"]
            else:
                height_in_cm = None
            
            #--to check the key city
            if "city" in v[i]:
                
                #--to check the child key city name
                if "name"in v[i]["city"]:
                    city_name = v[i]["city"]["name"]
                else:
                    city_name = None
                
                #--to check the child key city lat
                if "lat"in v[i]["city"]:
                    city_lat = v[i]["city"]["lat"]
                else:
                    city_lat = None
                
                #--to check the key city lon
                if "lon"in v[i]["city"]:
                    city_lon = v[i]["city"]["lon"]
                else:
                    city_lon = None
            else:
                city_name = None
                city_lat = None
                city_lon = None
            
            #--to check the key main_photo
            if "main_photo" in v[i]:
                main_photo = v[i]["main_photo"]
            else:
                main_photo = None
            
            #--to check the key pokemon_catch_rate
            if "pokemon_catch_rate" in v[i]:
                pokemon_catch_rate = v[i]["pokemon_catch_rate"]
            else:
                pokemon_catch_rate = None
            
            #--to check the key cats_owned
            if "cats_owned" in v[i]:
                cats_owned = v[i]["cats_owned"]
            else:
                cats_owned = None
            
            #--to check the key likes_cats
            if "likes_cats" in v[i]:
                likes_cats = v[i]["likes_cats"]
            else:
                likes_cats = None
            
            #--to check the key religion
            if "religion" in v[i]:
                religion = v[i]["religion"]
            else:
                religion = None
            
            #--creating list with single row data set
            data_ins_row =  [display_name,age,job_title,height_in_cm,city_name,city_lat,city_lon,main_photo,pokemon_catch_rate,\
                             cats_owned,likes_cats,religion]
            
            #--appending single row data set list with the main data set list
            data_ins_set.append(data_ins_row)

#print(" ")
#print(data_ins_set)

#--database transaction start

try:
    
    #--connecting the postgresql database server and postgresql database
    db_conn = pg.connect(host="localhost",database="postgres", user="postgres", password="postgres", port=5432)
    
    #--creating the cursor
    db_cur = db_conn.cursor()
    
    #--executing the table cleanup script
    db_cur.execute(del_sql_stm)
    
    #--executing the data insertion script
    db_cur.executemany(ins_sql_stm,data_ins_set)
    
    #--executing the data count check script
    db_cur.execute(cnt_sql_stm)
    total_inserted_row_count = db_cur.rowcount
    
    #--commiting the database changes
    db_conn.commit()
    
    ##-closing the cursor
    db_cur.close()
    
#--exception handling    
except (Exception, pg.DatabaseError) as error:
    
    #--showing the error statement
    print("Error : ",str(error))

#--to close the current database connection 
finally:
    if db_conn is not None:
        db_conn.close()

#--database transaction end

#--checking total inserted row count in the pandas dataframe
print(" ")
print("Total inserted row count : ",total_inserted_row_count)

#--comparing between incoming row count and inserted row count
print(" ")
if total_incoming_row_count == total_inserted_row_count:
    print("Success : All the rows are inserted from json data file to the database table successfully")
else:
    print("Error : All the row are not processed successfully. The count of missing rows are : ",\
          abs(total_incoming_row_count - total_inserted_row_count))


 
Total incoming row count :  25
 
Total inserted row count :  25
 
Success : All the rows are inserted from json data file to the database table successfully
